<a href="https://colab.research.google.com/github/vit050587/Natural_Language_Processing/blob/master/KVA_HW_11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#ДЗ. Урок 11. Модель Transformer
Взять предобученную трансформерную архитектуру и решить задачу перевода (для того же корпуса что вы выбрали из предыдущего дз)

In [1]:
!pip install transformers
!pip install transformers[sentencepiece]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 24.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.2 MB/s eta 0:00:00


In [2]:
import io
import re

In [3]:
# Download the file
path_to_file = "/content/drive/MyDrive/Colab Notebooks/Natural_Language_Processing/lesson10/rus-eng/rus.txt"

In [4]:
def preprocess_sentence(w):
  w = w.lower().strip()

  # creating a space between a word and the punctuation following it
  # eg: "he is a boy." => "he is a boy ."
  # Reference:- https://stackoverflow.com/questions/3645931/python-padding-punctuation-with-white-spaces-keeping-punctuation
  w = re.sub(r"([?.!,])", r" \1 ", w)
  w = re.sub(r'[" "]+', " ", w)

  # replacing everything with space except (a-z, A-Z, ".", "?", "!", ",")
  w = re.sub(r"[^a-zA-Zа-яА-Я?.!,']+", " ", w)

  w = w.strip()

  # adding a start and an end token to the sentence
  # so that the model know when to start and stop predicting.
  w = '<start> ' + w + ' <end>'
  return w

In [5]:
def create_dataset(path, num_examples):
  lines = io.open(path, encoding='UTF-8').read().strip().split('\n')

  word_pairs = [[preprocess_sentence(w) for w in l.split('\t')[:2]]  for l in lines[:num_examples]]

  return zip(*word_pairs)

In [7]:
en, ru = create_dataset(path_to_file, None)
print(en[20])
print(ru[20])

<start> duck ! <end>
<start> пригнись ! <end>


In [8]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("Helsinki-NLP/opus-mt-ru-en", do_lower_case=True)
model = AutoModelForSeq2SeqLM.from_pretrained("Helsinki-NLP/opus-mt-ru-en")

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


In [9]:
def translate(sentence):

  inputs = tokenizer.encode(sentence, return_tensors="pt")
  outputs = model.generate(inputs, max_length=40, num_beams=4, early_stopping=True)

  result = tokenizer.decode(outputs[0])

  return result

In [10]:
# хаотично беру предложения из корпуса с прошлого (10) домашнего задания.

for i in [5, 20, 50, 100, 256, 523, 526, 548, 4536, 62458, 46358, 1596]:
    print(ru[i], translate(ru[i]))

<start> хай . <end> <pad> <start> hey. <end></s>
<start> пригнись ! <end> <pad> <start> get down! <end></s>
<start> подождите . <end> <pad> <start> wait. <end></s>
<start> за ваше здоровье ! <end> <pad> For your health!</s>
<start> входи . <end> <pad> <start> come in. <end></s>
<start> отстань . <end> <pad> <start> leave. <end></s>
<start> будь храбр . <end> <pad> #start> be brave. <end></s>
<start> в наручники его . <end> <pad> <start> cuff him. <end></s>
<start> я был пьяный . <end> <pad> <start> I was drunk. <end></s>
<start> теперь не опаздывайте . <end> <pad> #start> Now don't be late. <end></s>
<start> я кормлю грудью . <end> <pad> #start> I breastfeed. <end></s>
<start> он большой ? <end> <pad> Is it big? <end></s>


На мой взгляд предобученная трансформерная архитектура справилась с переводом очень неплохо, за исключением этой строки " том слишком много говорит . volume speaks too much. <end" - это надо с именами собственными поработать.